<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/Machine-Learning/blob/main/ML/classes/class_march_3/class_march_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# REGRESIÓN LINEAL

## Objetivo:   

Predecir el costo de los autos de segunda mano en función de condiciones y/o caracteristicas del automovil que pueden afectar su precio de venta

#### Fuente: https://www.kaggle.com/datasets/mayankpatel14/second-hand-used-cars-data-set-linear-regression?resource=download

## Contenido Dataset: 

<ul>
<li>v.id</li>
<li>on road old</li>
<li>on road now</li>
<li>years</li>
<li>km</li>
<li>rating</li>
<li>condition</li>
<li>economy</li>
<li>top speed</li>    
<li>hp</li>
<li>torque</li>
<li>current price = Variable Respuesta</li>        
</ul>

## Importe de Paquetes

In [ ]:
### Tratamiento de Datos

import numpy as np
import pandas as pd

### Gráficos

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

### Preprocesado y modelado
from scipy.stats import pearsonr

# dividir en entrenamiento y testeo y realizar la validación
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# para la selección de modelos 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

# para el preprocesamiento de datos
from sklearn.preprocessing import StandardScaler


# para medir el desempeño de los modelos
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error



## Obtención de los datos y configuración del DataFrame 

In [ ]:
data = pd.read_csv('train.csv')

Realizó la obtención de datos constituyendo un DataFrame. 

In [ ]:
data.head(2) ### Primeros 05 registros del dataset

In [ ]:
data.tail(2) ### Ultimos 05 registros del dataset

Visualizó la parte inicial y final del DataFrame

Se recomienda realizar algunas observaciones verbales de la visualización con los métodos `.head()` y `.tail()`

In [ ]:
data.describe()

Se recomienda redactar observaciones verbales fruto de la visualización tabular de las variables cuantitativas con el método `describe()`

De la observación con el método `describe()` puede obtener:

* los atributos con datos faltantes
* la necesidad o no de escalamiento de los datos al observar los rangos de cada variable 

# Selección de algunos atributos

In [ ]:
datos= data[['current price','on road old', 'on road now', 'km',
       'top speed', 'hp', 'torque']]

In [ ]:
datos 

Se recomienda justificar la selección de los atributos predictores y la variable objetivo. 

Se recomienda configurar el Dataframe a la forma estándar $[X \mid y]$. 

In [ ]:
# Distribución de cada variable numérica

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(20, 10))
axes = axes.flat
columnas_numeric = datos.select_dtypes(include=['float64', 'int']).columns

for i, colum in enumerate(columnas_numeric):
    sns.histplot(data = datos, x = colum, stat = "count", kde = True, 
                 color = (list(plt.rcParams['axes.prop_cycle'])*2)[i]["color"],
                 line_kws= {'linewidth': 2}, alpha = 0.3, ax = axes[i])
    axes[i].set_title(colum, fontsize = 10, fontweight = "bold")
    axes[i].tick_params(labelsize = 6)
    axes[i].set_xlabel("")
    
    
fig.tight_layout()
plt.subplots_adjust(top = 0.9)
fig.suptitle('Distribución variables numéricas', fontsize = 10, fontweight = "bold");

Se recomienda documentar 

In [ ]:
datos_e= data[['years','economy', 'rating', 'condition']]

In [ ]:
# Distribución de cada variable numérica

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
axes = axes.flat
columnas_numeric = datos_e.select_dtypes(include=['float64', 'int']).columns

for i, colum in enumerate(columnas_numeric):
    sns.countplot(x = colum, data = datos_e, color = (list(plt.rcParams['axes.prop_cycle'])*2)[i]["color"], ax = axes[i])
    axes[i].set_title(colum, fontsize = 10, fontweight = "bold")
    axes[i].tick_params(labelsize = 6)
    axes[i].set_xlabel("")
    
    
fig.tight_layout()
plt.subplots_adjust(top = 0.9)
fig.suptitle('Distribución variables numéricas', fontsize = 10, fontweight = "bold");

In [ ]:
### Correlación de Variables

datacor = data.drop('v.id',axis=1) ## Eliminación de variable identificadora del vehiculo
corr = datacor.corr() ## Generación de correlación
plt.figure(figsize = (11,11))
sns.heatmap(corr, cbar = True, square = True, annot=True, fmt= '.2f',annot_kws={'size': 12}, cmap= sns.diverging_palette(20, 220, n=200))
plt.xticks(rotation = 45) ## Rotación de leyendas en x
plt.yticks(rotation = 45) ## Rotación de leyendas en y
plt.title("Coeficientes de Correlación", fontsize = 18, color = "black") ## Titulo del Gráfico

#### De acuerdo a lo anterior, observamos una correlación alta inversa entre el current price y el KM

#### División de los datos en Train y Test

In [ ]:
X = datacor.drop('current price', axis=1)
y = data['current price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [ ]:
## Validador de División

X_train.shape , X_test.shape

## Estandarización

In [ ]:
sc = StandardScaler()

### Estandarizando X Train
X_train_stand = sc.fit_transform(X_train)
X_train_stand_ok = pd.DataFrame(X_train_stand, columns = X_train.columns )

### Estandarizando X Test
X_test_stand = sc.transform(X_test)
X_test_stand_ok = pd.DataFrame(X_test_stand, columns = X_test.columns )

In [ ]:
X_train_stand

In [ ]:
X_train_stand_ok

In [ ]:
X_test_stand

In [ ]:
X_test_stand_ok

#### Modelado

In [ ]:
#### Regresión Lineal

reli = LinearRegression()
reli.fit(X_train_stand_ok, y_train)


In [ ]:
### Bosque Aleatorio

ratree = RandomForestRegressor()
ratree.fit(X_train_stand_ok,y_train)


In [ ]:
### Vecinos más cercanos

knei = KNeighborsRegressor()
knei.fit(X_train_stand_ok,y_train)

### Desempeño de los modelos

In [ ]:
def validate(result):
    ra = np.sqrt(- result)
    print ('score:' , ra)
    print ('Error Cuadratico Medio:', ra.mean())
    print ( 'Desviación Estandar:',ra.std())
    

In [ ]:
### Desempeño Regresión Lineal

desemp_regre = cross_val_score(reli, X_train_stand_ok,y_train, scoring='neg_mean_squared_error', cv =10 )
validate(desemp_regre)

In [ ]:
# Información del modelo Regresión Lineal

print("Intercept:", reli.intercept_)
print("Coeficiente:", list(zip(X_train_stand_ok.columns, reli.coef_.flatten(), )))
print("Coeficiente de determinación R^2:", reli.score(X_train_stand_ok, y_train))

In [ ]:
# Error de test del modelo Regresión Lineal

predicciones = reli.predict(X = X_test_stand_ok)
print(predicciones[0:10,])

rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print("")
print(f"El error cuadrático medio es: {rmse}")

In [ ]:
### Desempeño Bosque Aleatorio

desemp_ratree = cross_val_score(ratree, X_train_stand_ok,y_train, scoring='neg_mean_squared_error', cv =10 )
validate(desemp_ratree)

In [ ]:
### Error de test del modelo Bosque Aleatorio

y_pred = ratree.predict(X_test_stand_ok) 
print ('Error medio absoluto:', mean_absolute_error(y_test, y_pred))
print ('Error cuadrático medio:', mean_squared_error(y_test, y_pred))

In [ ]:
### Desempeño Vecinos más cercanos

desemp_knei = cross_val_score(knei, X_train_stand_ok,y_train, scoring='neg_mean_squared_error', cv =10 )
validate(desemp_knei)

In [ ]:
### Error de test del modelo Vecinos más cercanos

y_pred = knei.predict(X_test_stand_ok) 
print ('Error medio absoluto:', metrics.mean_absolute_error(y_test, y_pred))
print ('Error cuadrático medio:', metrics.mean_squared_error(y_test, y_pred))

Comparando los diferentes resultados, el valor que mayor se ajusta es una regresión lineal

# Referencias  

* El puntaje $R^{2}$: https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score


# 3.3.4.8. R² score, the coefficient of determination

La función `r2_score` calcula el coeficiente de determinación, generalmente indicado como $R^{2}$.

It represents the proportion of variance (of y) that has been explained by the independent variables in the model. 

It provides an indication of goodness of fit and therefore a measure of how well unseen samples are likely to be predicted by the model, through the proportion of explained variance.

As such variance is dataset dependent, R² may not be meaningfully comparable across different datasets. 

Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 

A constant model that always predicts the expected value of y, disregarding the input features, would get a $R^{2}$ score of 0.0.

If $\hat{y}_{i}$ is the predicted value of the -th sample and $y_{i}$ is the corresponding true value for total samples, the estimated $R^{2}$ is defined as:

$$ R^{2}(y, \hat{y}) = 1 - \frac{}{} $$